# 4 - pySAM run and desired outputs search



In [1]:
import PySAM
import PySAM.Pvsamv1 as PV
import PySAM.Grid as Grid
import PySAM.Utilityrate5 as UtilityRate
import PySAM.Cashloan as Cashloan
import pathlib, os
import json
import pvlib
import pandas as pd

In [2]:
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("Pvlib version: ", pvlib.__version__)
print("PySAM version: ", PySAM.__version__)

Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
Pvlib version:  0.10.4.dev9+ge92ee94
PySAM version:  5.1.0


In [3]:
jsonfolder = 'jsons'

jsonnames = ['vertical']

file_names = ["pvsamv1", "grid", "utilityrate5", "cashloan"]

pv4 = PV.new()  # also tried PVWattsSingleOwner
grid4 = Grid.from_existing(pv4)
ur4 = UtilityRate.from_existing(pv4)
so4 = Cashloan.from_existing(grid4, 'FlatPlatePVCommercial')


In [4]:
for count, module in enumerate([pv4, grid4, ur4, so4]):
    filetitle= 'vertical' + '_' + file_names[count] + ".json"
    with open(os.path.join(jsonfolder,filetitle), 'r') as file:
        data = json.load(file)
        for k, v in data.items():
            if k == 'number_inputs':
                continue
            try:
                if sys.version.split(' ')[0] == '3.11.7': 
                    # Check needed for python 3.10.7 and perhaps other releases above 3.10.4.
                    # This prevents the failure "UnicodeDecodeError: 'utf-8' codec can't decode byte... 
                    # This bug will be fixed on a newer version of pysam (currently not working on 5.1.0)
                    if 'adjust_' in k:  # This check is needed for Python 3.10.7 and some others. Not needed for 3.7.4
                        print(k)
                        k = k.split('adjust_')[1]
                module.value(k, v)
            except AttributeError:
                # there is an error is setting the value for ppa_escalation
                print(module, k, v)

adjust_constant
adjust_en_timeindex
adjust_en_periods
adjust_timeindex
adjust_periods
dc_adjust_constant
dc_adjust_en_timeindex
dc_adjust_en_periods
dc_adjust_timeindex
dc_adjust_periods


##### Sanity checks

In [5]:
pv4.SolarResource.solar_resource_file

'C:/Users/sayala/SAM Downloaded Weather Files/bellingham_washington_48.7525_-122.471_psm3-2-2-tmy_60_tmy.csv'

In [6]:
import _pickle as cPickle

with open(r"df_convert_219559.pkl", "rb") as input_file:
    df_weather_gid = cPickle.load(input_file)


with open(r"meta_convert_219559.pkl", "rb") as input_file:
    meta_dict = cPickle.load(input_file)

In [7]:
# pv4.Losses.en_snow_model = 0 #for snow losses to report.

#Need an except if it fails with snow bad snow depth 

In [8]:
pv4.unassign('solar_resource_file')

In [9]:
data = {'dn':list(df_weather_gid.dni),
       'df':list(df_weather_gid.dhi),
        'gh':list(df_weather_gid.ghi),
       'tdry':list(df_weather_gid.air_temperature),
       'wspd':list(df_weather_gid.wind_speed),
       'lat':meta_dict['latitude'],
       'lon':meta_dict['longitude'],
       'tz':meta_dict['timezone'],
       'elev':meta_dict['elevation'],
       'year':list(df_weather_gid.index.year),
       'month':list(df_weather_gid.index.month),
       'day':list(df_weather_gid.index.day),
       'hour':list(df_weather_gid.index.hour),
       'minute':list(df_weather_gid.index.minute),
       'alb':list(df_weather_gid.surface_albedo)}

pv4.value('solar_resource_data', data)

In [10]:
pv4.execute()
grid4.execute()
ur4.execute()
so4.execute()

# SAVE RESULTS|
results = pv4.Outputs.export()
economicresults = so4.Outputs.export()

In [11]:
#https://nrel-pysam.readthedocs.io/en/main/modules/Pvsamv1.html#PySAM.Pvsamv1.Pvsamv1.Outputs.annual_gh
# SOME THAT MIGHT BE RELEVANT.
results['annual_ac_gross']
results['annual_energy']
results['annual_gh']
#results['kwh_per_kw'] # Energy Yield
#results['performance_ratio'] # Energy Yield
#results['annual_snow_loss']
#results['monthly_snow_loss']
#results['subarray1_snow_coverage'] # Energy Yield
#economicresults['lcoe_nom']

1598863.0

In [12]:
df_weather_gid.ghi.sum()

1598863